H1 User Competency (Rasch Model)

In [1]:
# Versuche: nur Sätze nehmen, wo alle Interventiongruppen ungefähr gleichviel gemacht haben



# diffic_pro_user = xmlsaetze.groupby(['UserID','interventiongroup']).agg({'Schwierigkeit':'mean'}).reset_index()
# print(sns.boxplot(x=diffic_pro_user['Schwierigkeit'],y=diffic_pro_user["interventiongroup"],showfliers = False))

In [1]:
import pandas as pd
import pickle
from scipy import stats
import numpy as np
import seaborn as sns
import pingouin as pg
import statsmodels.formula.api as s

c:\Python310\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [22]:
#import
df = pickle.load(open( '../00_data/cleaned/sitzungssummary.pkl','rb') )
xmlsaetze = pickle.load(open( '../00_data/cleaned/xmlsaetze.pkl','rb') )

xmlsaetze = xmlsaetze[xmlsaetze.Testposition=='training']

rasch = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]

rasch['UserID'].astype(str)
rasch['SatzID'].astype(str)  

72780         4094
72783         4076
72786         4097
72791         4100
72798          375
            ...   
4980982    9991771
4980985    9991774
4981000    9991774
4981002    9991777
4981005    9991780
Name: SatzID, Length: 423426, dtype: object

In [23]:
test = rasch.groupby(['UserID','SatzID']).agg({'Datum':np.min})
test = test.reset_index()
test = pd.merge(test, rasch, on=['UserID','SatzID','Datum'])

SatzID,127,128,129,130,131,132,133,134,135,136,...,9992992,9992993,9992994,9992995,9992996,9992997,9992998,9992999,9993000,9993001
UserID,,,,,,,,,,,,,,,,,,,,,
186923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rasch = test.pivot_table(values='Erfolg', index=test.UserID, columns='SatzID', aggfunc='last')
rasch

In [6]:
#Subjects with only 1 valid response must be removed

In [26]:
delete_li = []
for col in rasch:
    cnt = rasch[col].count()
    if cnt < 2:
        delete_li.append(col)
        

In [11]:
rasch = rasch.drop(columns=[382, 3913, 9991884, 9992091])

In [27]:
delete_li = []
for index, row in rasch.iterrows():
    cnt = row.count()
    if cnt < 2:
        delete_li.append(index)
        rasch = rasch.drop([index])

rasch

SatzID,127,128,129,130,131,132,133,134,135,136,...,9992992,9992993,9992994,9992995,9992996,9992997,9992998,9992999,9993000,9993001
UserID,,,,,,,,,,,,,,,,,,,,,
186923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
452752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
rasch.to_csv('learning_data.csv')